In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink
from functions import (
    create_users_table, signup_user, login_user, create_map, generate_report, fetch_data,add_admin_user,create_population_buildings_risk_plot,fetch_and_insert_data
)

import pandas as pd
# Initialize database and create users table and admin user inserted
create_users_table()
add_admin_user()

# Output widget
output = widgets.Output()

# Login form
login_username = widgets.Text(description="Username:")
login_password = widgets.Password(description="Password:")
login_button = widgets.Button(description="Login")
login_message = widgets.Label()

# Sign-up form
signup_username = widgets.Text(description="Username:")
signup_password = widgets.Password(description="Password:")
signup_button = widgets.Button(description="Sign Up")
signup_message = widgets.Label()

def handle_signup(event):
    result = signup_user(signup_username.value, signup_password.value)
    signup_message.value = result
    if result == "User signed up successfully!":
        role = login_user(signup_username.value, signup_password.value)
        if role:
            clear_output()
            display_dashboard(role)
        else:
            signup_message.value = "Error in automatic login after signup!"

signup_button.on_click(handle_signup)
signup_form = widgets.VBox([signup_username, signup_password, signup_button, signup_message])

def handle_login(event):
    role = login_user(login_username.value, login_password.value)
    if role:
        clear_output()
        display_dashboard(role)
    else:
        login_message.value = "Invalid username or password!"

login_button.on_click(handle_login)

# Sign-up button to navigate to sign-up form
signup_nav_button = widgets.Button(description="Sign Up")

def show_signup_form(event):
    clear_output()
    display(signup_form)

signup_nav_button.on_click(show_signup_form)

login_form = widgets.VBox([login_username, login_password, login_button, login_message, signup_nav_button])

# Logout button
logout_button = widgets.Button(description="Logout")

def handle_logout(event):
    clear_output()
    display(login_form)

logout_button.on_click(handle_logout)

# Function to display the dashboard based on user role
def display_dashboard(role):
    clear_output()
    if role == 'admin':
        display(admin_dashboard)
    else:
        display(user_dashboard)


# Function to run the dashboard (existing dashboard functionality)
def run_dashboard(button=None):
    import requests
    import pandas as pd
    import geopandas as gpd
    import folium
    from branca.colormap import linear
    from ipywidgets import interact, Dropdown, Button, Output
    from IPython.display import FileLink, display

    # Load translation file and create a dictionary for header translation
    translation_df = pd.read_csv('Metadata_PIR_translation.csv')
    header_translation = dict(zip(translation_df['VARIABILE_INDICATORE'], translation_df['DEFINITIONS']))

    # Dropdown widget for selecting administrative level
    level_dropdown = Dropdown(
        options=['province', 'region', 'municipality'],
        value='province',
        description='Level:',
    )

    # Output widget for displaying map
    map_output = Output()
    plot_output = Output()

    # Function to update map and specific selection dropdown based on selected level
    def update_specific_dropdown(level):
        df = fetch_data(level)
        if level == 'province':
            options = df['provincia'].unique()
        elif level == 'region':
            options = df['regione'].unique()
        elif level == 'municipality':
            options = df['comune'].unique()
        
        specific_dropdown.options = options
        # Update the map with the entire level data
        with map_output:
            map_output.clear_output()
            m, map_filename = create_map(level)
            display(m)

    # Dropdown widget for selecting specific region, province, or municipality
    specific_dropdown = Dropdown(
        options=[],
        description='Specific:',
    )

    # Dropdown for selecting file format
    format_dropdown = Dropdown(
        options=['xlsx', 'csv'],
        value='xlsx',
        description='Format:',
    )

    # Button for generating report
    report_button = Button(description='Generate Report')

    # Function to handle report generation
    def handle_report_generation(b):
        filename = generate_report(level_dropdown.value, specific_dropdown.value, format_dropdown.value)
        with report_output:
            report_output.clear_output()
            display(FileLink(filename))

    report_button.on_click(handle_report_generation)

    # Output widget for displaying report generation message
    report_output = Output()

    # Function to update map based on selected level and specific selection
    def update_map(level, specific):
        with map_output:
            map_output.clear_output()
            m, map_filename = create_map(level, specific)
            display(m)
        with plot_output:
            plot_output.clear_output()
            create_population_buildings_risk_plot(level, specific)

    # Update specific dropdown when level changes
    level_dropdown.observe(lambda change: update_specific_dropdown(level_dropdown.value), names='value')

    # Update map when specific selection changes
    specific_dropdown.observe(lambda change: update_map(level_dropdown.value, specific_dropdown.value), names='value')

    # Display widgets and output
    display(level_dropdown, specific_dropdown, format_dropdown, report_button, report_output, map_output, plot_output)

    # Initial update of specific dropdown and map
    update_specific_dropdown(level_dropdown.value)

# Admin and User Dashboards
update_database_button = widgets.Button(description="Update Database")
view_dashboard_button_admin = widgets.Button(description="View Dashboard")

def update_database(button):
    with output:
        output.clear_output()
        try:
            exec(open("fetch_db_final.py").read())
            print("Database updated successfully!")
        except Exception as e:
            print(f"Error updating database: {e}")

update_database_button.on_click(update_database)
view_dashboard_button_admin.on_click(run_dashboard)

admin_dashboard = widgets.VBox([
    widgets.Label("Admin Dashboard: You can update the database here."),
    update_database_button,
    view_dashboard_button_admin,
    logout_button,
    output
])

view_dashboard_button_user = widgets.Button(description="View Dashboard")
view_dashboard_button_user.on_click(run_dashboard)

user_dashboard = widgets.VBox([
    widgets.Label("User Dashboard: You can view the dashboard here."),
    view_dashboard_button_user,
    logout_button
])

# Display initial login form
display(login_form)




    


Dropdown(description='Level:', options=('province', 'region', 'municipality'), value='province')

Dropdown(description='Specific:', options=(), value=None)

Dropdown(description='Format:', options=('xlsx', 'csv'), value='xlsx')

Button(description='Generate Report', style=ButtonStyle())

Output()

Output()

Output()